In [1]:
import mrcfile
import sys
sys.path.append('../analysis/')
from utils import *
import MDAnalysis
from MDAnalysis.analysis import align

nt_file = 'A.pdb'
solvmrc = 'Con2_1.8Acutoff.mrc'
molmap = 'Con2molmap.mrc'
mrc_f = '../maps/Con2-2.2A_sh.mrc'

C:\Users\racha\anaconda3\lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:

    # apply T to comp then R, reverse apply R and then T

In [11]:
import shutil
from tqdm import tqdm
# for each submap
# comp to ref --> 
# ref to comp --> R + T
def get_avg_dens(mrc_f,output,transforms,ref,Tref,swap=False):
    shutil.copy('../maps/Con2-2.2A_sh.mrc',output)
    dist_cutoff=5

    
    get_dens = get_density_interpolator(mrc_f)
    # get 2.2A coordinate frame
    with mrcfile.open(output, mode='r+') as mrc:
        xs = (np.arange(mrc.header.nxstart,mrc.header.mx)*mrc.voxel_size.x)+mrc.header.origin.x
        ys = (np.arange(mrc.header.nystart,mrc.header.my)*mrc.voxel_size.y)+mrc.header.origin.y
        zs = (np.arange(mrc.header.nzstart,mrc.header.mz)*mrc.voxel_size.z)+mrc.header.origin.z

        dist = np.absolute(np.subtract.outer(xs,ref.atoms.positions[:,0]))
        mindist = np.min(dist,axis=1)
        ok_xs = xs[mindist<dist_cutoff]
        dist = np.absolute(np.subtract.outer(xs,ref.atoms.positions[:,1]))
        mindist = np.min(dist,axis=1)
        ok_ys = ys[mindist<dist_cutoff]
        dist = np.absolute(np.subtract.outer(xs,ref.atoms.positions[:,2]))
        mindist = np.min(dist,axis=1)
        ok_zs = zs[mindist<dist_cutoff]
        #if swap:
        #    ok = ok_xs.copy()
        #    ok_xs = ok_zs.copy()
        #    ok_zs = ok
        for i,x in tqdm(enumerate(xs)):
            if x not in ok_xs:
                mrc.data[i,:,:] = 0
                continue
            for j,y in enumerate(ys):
                if y not in ok_ys:
                    mrc.data[i,j,:] = 0
                    continue
                for k,z in enumerate(zs):
                    mrc.data[i,j,k] = 0
                    if z in ok_zs and x in ok_xs and y in ok_ys:
                        dists = np.linalg.norm([x,y,z]-ref.atoms.positions,axis=1)
                        if dists.min()<dist_cutoff:
                            # all R,T
                            #if swap:
                            #    voxel = [z,y,x]
                            #else:
                            voxel = [x,y,z]
                            for R,T in transforms:
                                tvoxel = np.matmul(R,voxel-Tref) + T
                                #if swap:
                                #    tvoxel = np.flip(tvoxel)
                                dens = get_dens(tvoxel)
                                mrc.data[i,j,k] += dens/len(transforms)
# A
def get_all_nuc_dens(nt):
    ref = MDAnalysis.Universe(f'{nt}.pdb')
    u22 = MDAnalysis.Universe(PDB22_F)
    transforms = []
    backbone = ['OP1', "C2'", "O3'", "O5'", "O2'", "C3'", 'OP2', "C5'", "C4'", 'P', "C1'", "O4'"]
    atom_neam_ = list(set(ref.select_atoms(f'{RNA_SELECTION} and not ({" or".join(["name "+x for x in backbone])})').names))
    print(atom_neam_)
    atom_neam_.sort()
    ref_atoms = sum([ref.select_atoms(f'{RNA_SELECTION} and name {x}') for x in atom_neam_])
    Tref = ref_atoms.atoms.center_of_mass()
    for resnum in u22.select_atoms(f'resname {nt}').residues.resnums:
        if resnum!=22:
            comp_atoms = sum([u22.select_atoms(f'{RNA_SELECTION} and resnum {resnum} and name {x}') for x in atom_neam_])
            T = comp_atoms.atoms.center_of_mass() 
            R, rmsd = align.rotation_matrix(ref_atoms.positions - Tref,comp_atoms.positions - T)
            if rmsd < 1:
                transforms.append([R,T])                                
    get_avg_dens('../maps/Con2-2.2A_sh.mrc',f'{nt}_all.mrc',transforms,ref,Tref)
    get_avg_dens('../simulations/nt_wat.mrc',f'{nt}_md.mrc',transforms,ref,Tref)
    get_avg_dens('Con2_1.8Acutoff.mrc',f'{nt}_solv.mrc',transforms,ref,Tref,swap=True)
    get_avg_dens('Con2molmap.mrc',f'{nt}_molmap.mrc',transforms,ref,Tref,swap=True)

#
get_all_nuc_dens('C')
get_all_nuc_dens('G')
get_all_nuc_dens('U')
                                                                                   
get_all_nuc_dens('A')
# backbone later?

['O2', 'C2', 'C4', 'C6', 'C5', 'N1', 'N3', 'N4']


256it [00:59,  4.27it/s] 
256it [01:00,  4.24it/s] 
256it [01:43,  2.48it/s] 
256it [02:58,  1.43it/s] 


['C4', 'N3', 'N9', 'C6', 'C8', 'C5', 'O6', 'N7', 'N1', 'C2', 'N2']


256it [05:23,  1.26s/it]
256it [05:19,  1.25s/it]
256it [04:55,  1.15s/it]
256it [05:24,  1.27s/it]


['O2', 'C4', 'C6', 'C5', 'N1', 'N3', 'C2', 'O4']


256it [04:39,  1.09s/it]
256it [05:01,  1.18s/it]
256it [04:22,  1.03s/it]
256it [03:31,  1.21it/s]


['C4', 'N3', 'N9', 'C6', 'C8', 'C5', 'N6', 'N1', 'C2', 'N7']


256it [02:24,  1.77it/s]
256it [01:54,  2.23it/s] 
256it [01:58,  2.17it/s]
256it [02:01,  2.10it/s] 


In [89]:
ref_atoms.positions

array([[106.24 , 108.789,  97.701],
       [109.821, 107.453,  99.779],
       [105.374, 107.594,  97.991],
       [104.017, 108.232,  98.19 ],
       [108.05 , 108.719,  99.538],
       [104.051, 109.327,  97.14 ],
       [108.343, 109.467, 100.638],
       [103.104, 110.463,  97.389],
       [109.497, 109.148, 101.346],
       [106.532, 110.233,  99.853],
       [110.215, 108.115, 100.871],
       [108.747, 107.687,  99.049],
       [109.912, 109.796, 102.425],
       [107.365, 110.441, 100.833],
       [106.887, 109.214,  99.033],
       [105.363, 106.681,  96.891],
       [102.958, 107.31 ,  97.954],
       [105.386, 109.8  ,  97.172],
       [103.351, 111.022,  98.64 ],
       [102.615, 112.361,  99.078]], dtype=float32)

array([[ 97.701, 108.789, 106.24 ],
       [ 99.779, 107.453, 109.821],
       [ 97.991, 107.594, 105.374],
       [ 98.19 , 108.232, 104.017],
       [ 99.538, 108.719, 108.05 ],
       [ 97.14 , 109.327, 104.051],
       [100.638, 109.467, 108.343],
       [ 97.389, 110.463, 103.104],
       [101.346, 109.148, 109.497],
       [ 99.853, 110.233, 106.532],
       [100.871, 108.115, 110.215],
       [ 99.049, 107.687, 108.747],
       [102.425, 109.796, 109.912],
       [100.833, 110.441, 107.365],
       [ 99.033, 109.214, 106.887],
       [ 96.891, 106.681, 105.363],
       [ 97.954, 107.31 , 102.958],
       [ 97.172, 109.8  , 105.386],
       [ 98.64 , 111.022, 103.351],
       [ 99.078, 112.361, 102.615]], dtype=float32)

In [20]:
rmsd

0.23011942654472267

In [ ]:
        md_coords = np.matmul(R,atom_coord) + T
        all_mg_dens.append(md_mg_dens(md_coords)[0])